# Getting Started with AWS Bedrock Agent Library

This notebook demonstrates how to use the bedrock_agent library from the src/utils directory to create and interact with AWS Bedrock Agents. This library provides high-level Pythonic objects such as Agent, Guardrail, and Tool to simplify building with Bedrock Agents in Python.

## Importing the library

In [13]:
import os
import sys
sys.path.append('../..')
sys.path.append('../../src')
from utils.bedrock_agent import Agent, Tool, ParameterSchema, ParamType, Toolbox, Task

## Basic Agent operations - create, test existence, invoke, delete

Let's start by creating a simple 'Hello World' agent. This agent will demonstrate the basic components needed to create and interact with a Bedrock Agent.

### Simple create and invoke

In [9]:
Agent.set_force_recreate_default(True)
agent = Agent.create("poet-agent", "Write poems", "Write poems as directed by the user", "write poems")
print(agent.invoke("Please write a poem about ice hockey"))


Deleting existing agent and corresponding lambda for: poet-agent...
Agent poet-agent not found
Creating agent poet-agent...
Created agent, id: 3EKQCOYQHO, alias id: TSTALIASID

Waiting for agent status to change. Current status CREATING
Agent id 3EKQCOYQHO current status: NOT_PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id 3EKQCOYQHO current status: PREPARED
DONE: Agent: poet-agent, id: 3EKQCOYQHO, alias id: K0IOCRX6CZ

Here's a poem about ice hockey:

Blades of Steel

On frozen canvas, warriors glide,
Swift as wind, with sticks at side.
Puck flies fast, a rubber missile,
Players chase with strength and grit.

Boards rattle, ice sprays white,
Bodies clash in friendly fight.
Goalies stand, the last defense,
Guarding nets with focus intense.

Teamwork flows like graceful dance,
Each pass a risk, each shot a chance.
Cheers erupt as lamp is lit,
Glory found in hockey's spirit.

In winter's chill, a game so bold,
Where legends rise and stories told.
On sacre

### Check for the agent's existence
Note that the agent is persisted in Bedrock:

In [10]:
print(Agent.exists("poet-agent"))

True


### Delete the agent

In [11]:
Agent.delete_by_name("poet-agent")

Deleting alias K0IOCRX6CZ from agent 3EKQCOYQHO
Deleting alias TSTALIASID from agent 3EKQCOYQHO


## More advanced Agent creation
You can specify more information when creating an Agent, and can create them in other ways, such as from a YAML configuration file. Alteratively, you can also modify the agent (ex. attach a Tool) after its creation.

In [14]:
calculator_agent = Agent.create(name="calculator_agent",
                     role="Perform calculations",
                     goal="Calculate responses to provided questions",
                     instructions="""If the question involves a numerical calculation, execute it using the 
                                     code interpreter as needed. If it is not a calculation, reply with the
                                     phrase 'I can help you with calculations. Do you have a calcalation for me?
                                     """,
                     llm="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
                     code_interpreter=True,  # lets us write code to do accurate calculations
                     verbose=True)
print(calculator_agent.invoke("What is sin(.268) * pi/2?"))




Deleting existing agent and corresponding lambda for: calculator_agent...
Agent calculator_agent not found
Creating agent calculator_agent...
Creating agent: calculator_agent...
Created agent IAM role: arn:aws:iam::271976243307:role/DEFAULT_AgentExecutionRole...
Creating agent: calculator_agent with model: us.anthropic.claude-3-5-sonnet-20241022-v2:0...
kwargs: {}
Created agent, resulting id: IUDRB8FTP4
{'ResponseMetadata': {'RequestId': 'a0d6b55f-5902-44ff-8e98-5a1a8d3a1215', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 25 Jan 2025 04:01:19 GMT', 'content-type': 'application/json', 'content-length': '14365', 'connection': 'keep-alive', 'x-amzn-requestid': 'a0d6b55f-5902-44ff-8e98-5a1a8d3a1215', 'x-amz-apigw-id': 'E7RCfFIZvHcEfhQ=', 'x-amzn-trace-id': 'Root=1-6794620f-79b06d907ba9848b22d48af8'}, 'RetryAttempts': 0}, 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:271976243307:agent/IUDRB8FTP4', 'agentCollaboration': 'DISABLED', 'agentId': 'IUDRB8FTP4', 'agentName': 'calculato

### Creating an Agent from a YAML template
It is often convenient, particularly when working with multiple agents, to define the agents in a YAML file. The file can contain multiple agent definitions and is, by default, 'agents.yaml' (although you can specify a different filename):

In [2]:
Agent.set_force_recreate_default(True)
test_agent = Agent.create_from_yaml('test_agent')


Deleting existing agent and corresponding lambda for: test_agent...
Found target agent, name: test_agent, id: IB3WJWSBFK
Deleting aliases for agent IB3WJWSBFK...
Deleting alias TSTALIASID from agent IB3WJWSBFK
Deleting alias W0JDTUVBJ2 from agent IB3WJWSBFK
Deleting agent: IB3WJWSBFK...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_test_agent...
Creating agent test_agent...
Created agent, id: PIQSC2FAHG, alias id: TSTALIASID

Waiting for agent status to change. Current status CREATING
Agent id PIQSC2FAHG current status: NOT_PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id PIQSC2FAHG current status: PREPARED
DONE: Agent: test_agent, id: PIQSC2FAHG, alias id: XWJNB3KT3S



Tha YAML file contains definitions: 

## Working with Tools

Agents can be enhanced with tools that allow them to perform specific actions. Tools can be attached at Agent create time or later. They can be retrieved from the Toolbox (existing pre-registered Tools backed by lambdas), created from local code files, or created directly from Python functions.

### Attaching tools from the toolbox
Tools that are already registered in the Toolbox can be attached to an agent. You can add Tools to the Toolbox via the Toolbox class or the `tool_manager.py` in the `shared` directory (see the README.md for the tool manager).

In [8]:
if Toolbox.is_registered("web_search"):
    test_agent.attach_tool_by_name("web_search")
    print(test_agent.invoke("What is recent news for AMZN?"))
else:
    print("Register the web search tool first")

boto3 version: 1.35.79
Here are the recent news highlights for Amazon (AMZN):

1. Amazon is expanding its same-day delivery service to more cities, enhancing its logistics capabilities.
2. The company is facing increased scrutiny from regulators over its business practices and market dominance.
3. Amazon's cloud computing division, AWS, continues to be a major revenue driver for the company.
4. The e-commerce giant is investing heavily in artificial intelligence and machine learning technologies.
5. Amazon is making efforts to improve its environmental sustainability, including initiatives to reduce packaging waste and increase the use of renewable energy in its operations.


### Creating tools from code
You can create a Tool from a Python function that is annotated with type hints. Note this currently materializes a lambda created from the function:

In [3]:
def mask_string(input_string: str) -> str:
    """Masks a string by replacing all but the last four characters with asterisks."""
    if len(input_string) <= 4:
        return input_string
    else:
        return "*" * (len(input_string) - 4) + input_string[-4:]

In [5]:
test_agent.attach_tool_from_function(mask_string)
test_agent.invoke("Please mask this string: 077-29-2932")

'The masked string is: XXX-XX-2932'

### Creating a Tool via a ParameterSchema
If you have a lambda that is not in Python and/or does not have type hints, you can create a ParameterSchema for it to define the parameters needed. From that and a code file, you can create a Tool, and then attach it to your Agent.

In [7]:
schema = ParameterSchema.create_with_values(name="input_string",
                                            parameter_type=ParamType.STRING,
                                            description="String to transmorgify",
                                            required=False)
transmorgifier_tool = Tool.create("transmorgifier",
                                        schema=schema,
                                        code_file="lambda_transmorgify_string.py",
                                        description="transmorgifies a string")
test_agent.attach_tool(transmorgifier_tool)
print(test_agent.invoke("Please transmorgify this string: Who can say what the rain may bring?"))

The transmorgified string is: Jub pna fnl jung gur enva znl oevat?


### Defining Tools at Agent creation
You can define the Tool with the agent, if its parameter schema is available:

In [26]:
test_user_data_agent = Agent.create(
    name="test_user_data_agent",
    role="Accesser of user data",
    goal="Find user profile data and return it",
    instructions="Use the provided tool to find user profile data and return it",
    tool_code="lambda_lookup_user_profile.py",
    tool_defs=[
        {
            "name": "lookup_user_profile",
            "description": "Return user profile data as a JSON structure. Requires an input user id",
            "parameters": {
                "user_id": {
                    "description": "The unique identifier of the user",
                    "type": "string",
                    "required": False
                }
            }
        }
    ],
    llm="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    code_interpreter=False,
    verbose=True
)


Deleting existing agent and corresponding lambda for: test_user_data_agent...
Agent test_user_data_agent not found
Creating agent test_user_data_agent...
Creating agent: test_user_data_agent...
Created agent IAM role: arn:aws:iam::271976243307:role/DEFAULT_AgentExecutionRole...
Creating agent: test_user_data_agent with model: us.anthropic.claude-3-5-sonnet-20241022-v2:0...
kwargs: {}
Created agent, resulting id: 7W5P4PWUDW
{'ResponseMetadata': {'RequestId': 'ca19d7f4-e9a9-4439-82cb-8e8c944b7432', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 25 Jan 2025 04:25:25 GMT', 'content-type': 'application/json', 'content-length': '14053', 'connection': 'keep-alive', 'x-amzn-requestid': 'ca19d7f4-e9a9-4439-82cb-8e8c944b7432', 'x-amz-apigw-id': 'E7UkcEVCvHcEVPw=', 'x-amzn-trace-id': 'Root=1-679467b5-3f1a501200ede9e025cf09c0'}, 'RetryAttempts': 0}, 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:271976243307:agent/7W5P4PWUDW', 'agentCollaboration': 'DISABLED', 'agentId': '7W5P4PWUDW', 'ag

In [28]:
Agent.delete_by_name("test_user_data_agent")

Deleting alias TSTALIASID from agent 7W5P4PWUDW
Deleting alias ZPQIHIVQGZ from agent 7W5P4PWUDW


## Working with Knowledge Bases

Agents can be enhanced with knowledge bases to provide additional context and information.

In [ ]:
# Create a knowledge base (to show attachment to an Agent on creation) 
import time
from src.utils.knowledge_base_helper import KnowledgeBasesForAmazonBedrock
import random
# Create a quick KB for demonstration purposes
kb_helper = KnowledgeBasesForAmazonBedrock()
kb_name = "mortgage-test-kb"
random_string = ''.join(random.choices('0123456789', k=4))
bucket_name = f"test-kb-bucket-{random_string}"
kb_id, ds_id = kb_helper.create_or_retrieve_knowledge_base(
        kb_name,
        kb_description="Useful for answering questions about mortgage refinancing and for questions comparing various mortgage types",
        data_bucket_name=bucket_name
)
# Normally you would upload data to the bucket here

time.sleep(30)  # ensure that the kb is available
kb_helper.synchronize_data(kb_id, ds_id)  # sync knowledge base

### Attach a knowledge base on agent creation

In [ ]:
# Create an agent, specifying the LLM and attaching a KnowledgeBase
general_mortgage_questions = Agent.create(
        name="mortgage_test_agent",
        role="General Mortgage Questions",
        goal="Handle conversations about general mortgage questions, like high level concepts of refinancing or tradeoffs of 15-year vs 30-year terms.",
        instructions="""You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types.""",
        kb_id=kb_id,
        kb_descr="""Use this knowledge base to answer general questions about mortgages, like how to refinance, or the difference between 15-year and 30-year mortgages.""",
        llm="us.anthropic.claude-3-5-sonnet-20241022-v2:0"
)

### Clean up the knowledge base

In [ ]:
kb_helper.delete_kb("mortgage_test_kb")

## Using Tasks

Tasks provide a structured way to interact with agents. The expected_output allows reviewer agents to assess whether the task is functionally complete.

In [ ]:
# Create some tasks
task1 = Task.create("greet", "Generate a friendly greeting", expected_output="A warm greeting")
task2 = Task.create("translate", "Translate the greeting to Spanish", expected_output="A Spanish greeting")

# Invoke the agent with multiple tasks
result = test_agent.invoke_with_tasks([task1, task2])
print(result)

## Cleanup
Run this cleanup code to delete the agents created above:

In [17]:
Agent.delete_by_name("mortgage_test_agent")
Agent.delete_by_name("test_agent")
Agent.delete_by_name("calculator_agent")

Agent mortgage_test_agent not found
Deleting alias TSTALIASID from agent PIQSC2FAHG
Deleting alias XWJNB3KT3S from agent PIQSC2FAHG
Deleting alias F16P3JPD8G from agent IUDRB8FTP4
Deleting alias TSTALIASID from agent IUDRB8FTP4
